In [1]:
import cv2
import numpy as np
import os
import pytesseract as pt

In [2]:
#setting
INPUT_WIDTH = 640
INPUT_HEIGHT = 640

In [3]:
#LOAD THE IMAGE
img= cv2.imread('./Module_4_Yolo_v5/test_images/getty_sample.jpg')

cv2.namedWindow('test image',cv2.WINDOW_KEEPRATIO)
cv2.imshow('test image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## LOAD YOLO MODEL

In [4]:

net = cv2.dnn.readNetFromONNX('./Model2-20240626T003829Z-001/Model2/weights/best.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)


In [5]:
#CONVERT THE IMAGE INOT YOLO FORMAT
def get_detections(img,net):
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image
    
    #GET PREDICTIONS FROM YOLO MODEL
    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB=True,crop=False)
    net.setInput(blob)
    preds= net.forward()
    detections = preds[0]
    
    return input_image, detections

#cv2.namedWindow('test image',cv2.WINDOW_KEEPRATIO)
#cv2.imshow('test image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()






def non_maximum_supression(input_image,detections):

#FILTER DETECTION BOXES BASED ON CONFIDENCE AND PROBABILTY SCORE
    boxes=[]
    confidences =[]

    image_w,image_h= input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):
        row=detections[i]
        confidence=row[4]
        if confidence>0.4:
            class_score=row[5]
            #print(class_score)
            if class_score>0.25:
                cx, cy, w, h =row[0:4]
               # print(cx,cy,w,h)

                left=int((cx - 0.5*w)*x_factor)
                top =int((cy - 0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box =np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    #clean           
    boxes_np=np.array(boxes).tolist()
    confidences_np=np.array(confidences).tolist()

    #NMS
    #index= cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()
    index = np.array(cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)).flatten()
    
    return boxes_np, confidences_np, index



#drawing
def drwaings(image,boxes_np,confidences_np,index):
    for ind in index:
        x,y,w,h = boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)


        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)

        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)

    return image



In [6]:
#predictions
def yolo_predections(img,net):
    ##Step1: detetctions
    input_image, detections = get_detections(img,net)

    ##Step2:NMS(non maximum sepration)
    boxes_np, confidences_np, index = non_maximum_supression(input_image,detections)


    ##Step3 Drawing
    result_img = drwaings(input_image,boxes_np,confidences_np,index)
    return result_img

In [ ]:
#testing
#img= cv2.imread('./Module_4_Yolo_v5/test_images/getty_sample.jpg')

results = yolo_predections(img,net)

In [ ]:
cv2.namedWindow('result',cv2.WINDOW_KEEPRATIO)
cv2.imshow('result',results)
cv2.waitKey(0)
cv2.destroyAllWindows()

## REAL TIME OBJECT DETECTION

In [ ]:
import cv2

def yolo_predictions(frame, net):
    # Assuming this function processes the frame and returns the processed frame
    # Add the actual implementation of yolo_predictions function here
    return frame  # Placeholder, replace with actual processing code

cap = cv2.VideoCapture('./test_images/traffic2.mp4')

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    
    if not ret:
        print('Unable to read video or end of video reached')
        break
    
    results = yolo_predictions(frame, net)
    
    if results is None:
        print('Error: YOLO predictions returned None')
        break
    
    cv2.namedWindow('YOLO', cv2.WINDOW_KEEPRATIO)
    cv2.imshow('YOLO', results)
    
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()